In [ ]:
import os
import cv2 as cv2
import numpy as np
import pickle
import pandas as pd
import csv
import collections
from moviepy.editor import *

In [ ]:
def integration(frame,threshold):
    # 'frame' is the recorded frame after using pixel to detect cutting frame
    # 'threshold' is a standard time to integrate very close frames
    
    length = len(frame)
    cutting_frame = []
    arr = 0
    
    while arr < length:
        brr = 0
        if arr + brr == length -1:
            arr += 1
        
        while brr < length - arr - 1:
            if frame[arr+brr+1] - frame[arr+brr] <= threshold:
                brr +=1
                
                if arr + brr + 1 == length:
                    cutting_frame.append(frame[arr])
                    arr = length
                
            else:
                cutting_frame.append(frame[arr])
                arr = arr + brr + 1
                break
    cutting_frame.append(frame[-1])
    return cutting_frame

In [ ]:
def video_clip(cutting_frame,load_path,clip_path,fps):
    length1 = len(cutting_frame)
    pair_list = []
    for i in range(length1-1):
        if i == 0:
            pair_list.append([cutting_frame[i],cutting_frame[i+1]-1])
        else:
            pair_list.append([cutting_frame[i]+2,cutting_frame[i+1]-1])
    
    length2 = len(pair_list)
    for j in range(length2):
        video_clip = VideoFileClip(load_path).subclip(pair_list[j][0]/fps,pair_list[j][1]/fps)
        
        video_clip.write_videofile(clip_path +'/'+str(j)+'.mp4',fps,codec='libx264',audio=False)
    
    return pair_list

In [ ]:
"""
This function is to create the cutting pair list for fading away video
"""
def FadingVideo_clip(start_list,end_list,frame_Num,load_path,clip_path,fps):
    length = len(start_list)
    pair_list = []
    
    pair_list.append([0,start_list[0]-1])
    
    for i in range(length-1):
        pair_list.append([end_list[i]+2,start_list[i+1]-1])
    pair_list.append([end_list[-1]+2,frame_Num])
    
    length2 = len(pair_list)
    for j in range(length2):
        video_clip = VideoFileClip(load_path).subclip(pair_list[j][0]/fps,pair_list[j][1]/fps)
        video_clip.write_videofile(clip_path+'/'+str(j)+'.mp4',fps,codec='libx264',audio=False)
    
    return pair_list

In [ ]:
"""
functions to save documents as .plk
"""
def save_obj(data, name):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
"""
functions to save start time and end time as csv file
"""
def write_csv(pair_list,clip_path,fps,name):
    pd_data = []
    for i in range(len(pair_list)):
        pd_data.append([str(i),pair_list[i][0],pair_list[i][1],pair_list[i][0]/fps,pair_list[i][1]/fps])
        
    df = pd.DataFrame(data=pd_data,columns = ['Filename','start_frame','end_frame','start_time','end_time'])
    df.to_csv (clip_path+'/'+name+'.csv', encoding = "utf-8")

In [ ]:
"""
fuction to read video frame by frame
"""
def read_video(load_path):
    vid_name = load_path 
    vidcap = cv2.VideoCapture(vid_name)
    frame_num = 0
    hist_gray = []

    while True:
        success,frame = vidcap.read()
        if not success:
            print ("can't read video "+ vid_name ) 
            break
        if frame_num % 500 == 0:
            print(frame_num)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray],[0],None,[256],[0,256])
       #hist = cv2.calcHist([gray[300:,:600]],[0],None,[256],[0,256])
        hist_gray.append(hist)
        
        frame_num += 1

        k = cv2.waitKey(33)
        if k==27:    # Esc key to stop
            break
        elif k==-1:  # normally -1 returned,so don't print it
            continue

    vidcap.release()
    #out.release()
    cv2.destroyAllWindows()
    
    return hist_gray,frame_num

In [ ]:
"""
Function to detect switching frame
"""
def switching_frame(hist_gray,threshold,frame_num):
    score = []
    hist_list = [0]
    for i in range(len(hist_gray)-1):
        dif = hist_gray[i] - hist_gray[i+1]
        compare = np.sum(np.abs(dif))/256
        score.append(compare)
        if compare > threshold:
            hist_list.append(i)
            print(i)
    hist_list.append(frame_num)
    print('Length of independent video clip')
    print(len(hist_list))
    
    return hist_list,score

# This part is to process video without fading effect

In [ ]:
"""
This part is to process video without fading effect and setting parameters and path
"""
fps = 30      # frame per second
load_path = '' # load video from local path
hist_threshold = 500 # grey level histogram threshold, recording frames of which grey level histogram changes
                     # exceed this threshold as potential transition frames. try 250,500,1000...
integration_threshold = 30 # if two recorded frames happens within this range, treat them as one scene, and 
                           # no trasition happens.
clip_path = '' # Save path

In [1]:
def read_images(im_dir):
    
    frame_num = 0
    hist_gray = []

    while True:
        success,frame = vidcap.read()
        if not success:
            print ("can't read video "+ vid_name ) 
            break
        if frame_num % 500 == 0:
            print(frame_num)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray],[0],None,[256],[0,256])
       #hist = cv2.calcHist([gray[300:,:600]],[0],None,[256],[0,256])
        hist_gray.append(hist)
        
        frame_num += 1

        k = cv2.waitKey(33)
        if k==27:    # Esc key to stop
            break
        elif k==-1:  # normally -1 returned,so don't print it
            continue

    vidcap.release()
    #out.release()
    cv2.destroyAllWindows()
    
    return hist_gray,frame_num

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [ ]:
[hist_gray,frame_num] = read_video(load_path)

In [ ]:
[hist_list,score] = switching_frame(hist_gray,hist_threshold,frame_num)

In [ ]:
cutting_frame = integration(hist_list,integration_threshold)
cutting_frame,len(cutting_frame)

In [ ]:
pair_list = video_clip(cutting_frame,load_path,clip_path,fps)